In [1]:
import duckdb

In [7]:
duckdb.sql("""
create or replace table malware as
           select * from read_csv(Software_Malware_Detection_Data.csv)
""")

In [25]:
duckdb.sql("""
           with cte1 as
           (select *,
           row_number()over(partition by software_id order by rundate::date desc) as rn
            from malware)
           select software_id,today,today-yesterday as difference_to_previous
           
           from 
                
                (select software_id,
                sum(case when rn=1 then malware_detected end) as today,
                sum(case when rn=2 then malware_detected end) as yesterday
                    from cte1
                group by software_id
                having today>yesterday and today>11 and count(*)>2
                order by software_id desc,today desc)
           
           """)

┌─────────────┬────────┬────────────────────────┐
│ software_id │ today  │ difference_to_previous │
│   varchar   │ int128 │         int128         │
├─────────────┼────────┼────────────────────────┤
│ SFT009      │     21 │                      3 │
│ SFT007      │     24 │                     16 │
│ SFT005      │     16 │                      4 │
│ SFT004      │     22 │                     13 │
│ SFT003      │     18 │                      5 │
│ SFT002      │     18 │                      5 │
│ SFT001      │     13 │                      6 │
└─────────────┴────────┴────────────────────────┘